# Import Dependencies

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st

/var/folders/ms/lv6zlm_16wx8dzf7c_t51b0r0000gn/T/ipykernel_73195/426100267.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


# Parse First Text File

In [2]:
file_path = 'mathwell_questions2.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df1 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df1)/total_questions)*100) #past percentage was 54.88
print(result1)
print(len(df1), total_questions)

0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
Percentage of Questions with Executable Code: 55.54768754149148
12551 22595


In [3]:
print(f"Percentage of questions that have 0 as answer: {(len(df1[df1['answer']==0])/len(df1))*100}")

Percentage of questions that have 0 as answer: 6.915783602900167


In [4]:
df1 = df1.drop_duplicates(subset='question')

In [5]:
df1

,question,solution,answer,topic
0,A baker baked 120 cakes. He sold 40 on the fir...,def solution():\n\n # The baker started wit...,30,Animals and Wildlife
1,Naruto has 300 ramen noodles. He eats 20 ramen...,def solution():\n #Naruto has 300 ramen noo...,200,Naruto
2,A 5th-grade class is doing a science project. ...,def solution():\n #20 students are in the c...,60,DIY Science Projects
3,A mummy is 3000 years old. \n\nHis sarcophagus...,def solution():\n #The mummy is 3000 years ...,4000,Mummies
4,The volleyball team has 12 players. \n4 of the...,def solution():\n #Total number of players\...,0,volleyball
...,...,...,...,...
12546,The Justice League has 7 members. \n\n2 of the...,def solution():\n #The Justice League has 7...,5,The Justice League
12547,Sarah has 12 friends. \n\n3 of her friends are...,def solution():\n #Sarah has 12 friends\n ...,0,Friendship and Relationships
12548,There are 2000 vampires in a small town. 1/10 ...,def solution():\n #Total vampires\n vamp...,4000.0,Myths and Legends
12549,A band has 400 instruments. 100 of them are gu...,def solution():\n #Total instruments\n t...,150,Music Instruments


# Parse Second Text File

In [6]:
file_path = 'mathwell_questions_no_topic.txt'
data = []

with open(file_path, 'r') as file:
    text = file.read()
    sections = text.split("Topic: ")
    #sections = text.split("### Response:")
    del sections[0]  # Remove the initial part before the question
    total_questions = len(sections)
    for section in sections:
        try:
            topic = section.split("### Response:")[0].strip()
            section = section.split("### Response:")[1].strip()
            question = section.split("Question:")[1].strip()
            question = question.split("Solution:")[0].strip()
            solution = section.split("Solution:")[1].strip()
            if "##" in question:
                question = question.split("##")[0]
#             if "\n\n" in question:
#                 question = question.split("\n\n")
#                 question = "\n".join(question)
            if "\nBel" in solution:
                solution = solution.split("\nBel")[0]
            if "##" in solution:
                solution = solution.split("##")[0]
            if "return" not in solution:
                try: 
                    solution = solution + '\n' + '    return result'
                    solution_text = solution
                    try:
                            # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, 'topic': topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                                pass
                except:
                    pass
            if "return" in solution:
                try: 
                    solution_text = solution
                    try:
                        # Execute the Python code
                        exec(solution)
                        answer = solution()
                        if answer==None:
                            continue
                        data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                    except: 
                        try: #pick up questions that did not indent the code block
                            solution_new = ""
                            lines = solution.split('\n')
                            indent = False
                            for line in lines:
                                if line.startswith('def solution():'):
                                    solution_new += line + '\n'

                                if not line.startswith('def solution():') and not line.startswith('    return result'):
                                    solution_new += '    ' + line + '\n'

                                if line.startswith('    return result'):
                                    solution_new += line
                            solution_text = solution_new
                            # Execute the Python code
                            exec(solution_new)
                            answer = solution()
                            if answer==None:
                                continue
                            data.append({"question": question, "solution": solution_text, 'answer': answer, "topic": topic})
                        except:
                            pass
                except:
                    pass
        except:
            pass
df2 = pd.DataFrame.from_dict(data)
result1 = "Percentage of Questions with Executable Code: " + str((len(df2)/total_questions)*100)
print(result1)
print(len(df2), total_questions)

Percentage of Questions with Executable Code: 46.60845662543455
5765 12369


In [7]:
print(f"Percentage of questions that have 0 as answer: {(len(df2[df2['answer']==0])/len(df2))*100}")

Percentage of questions that have 0 as answer: 4.9609713790112755


In [8]:
df2 = df2.drop_duplicates(subset='question')

In [9]:
df2

,question,solution,answer,topic
0,2000 people in a stadium. 1/4 of the people ar...,def solution():\n #Total people in the stad...,850.0,unspecified
1,The first 500 people in line for the Comic-Con...,def solution():\n #The first 500 people in ...,1000,unspecified
2,There are 120000 spiders in a spider farm. 100...,def solution():\n #There were 120000 spider...,100000,unspecified
3,A baker bakes 100 pies. He sells 20 of them on...,def solution():\n #Baker bakes 100 pies\n ...,10,unspecified
4,A 747 can hold 400 people. \nA 777 can hold 30...,def solution():\n #A 747 can hold 400 peopl...,4000,unspecified
...,...,...,...,...
5758,1000 people are at a rally. 200 of them are we...,def solution():\n #1000 people are at a ral...,600,unspecified
5759,A 10-gallon fish tank has 100 small fish. 20 o...,def solution():\n #10-gallon fish tank has ...,110,unspecified
5760,"In the first year, there were 500 students at ...",def solution():\n #There were 500 students ...,50,unspecified
5761,"The 2020 Olympics in Tokyo, Japan, has 12000 a...",def solution():\n #Total number of athletes...,3000,unspecified


# Concat DFs

In [10]:
extra_qs = pd.read_csv('data/extra_qs.csv')
df = pd.concat([df1, df2, extra_qs])
df = df.drop_duplicates(subset='question')
df = df.sample(frac = 1, random_state = 42)
df = df[['question', 'solution', 'answer', 'topic']]
df.to_csv("data/sgsm_unannotated.csv")

In [11]:
df ## last run had 16245

,question,solution,answer,topic
10074,"Ares, the God of War, has 1000 swords. He uses...",def solution():\n #Ares started with 1000 s...,480.0,Mythology
6288,The Solar System has 8 planets. \n5 of the pla...,def solution():\n #Total planets\n total...,0,The Solar System
8509,A circus has 12 clowns. 3 of the clowns are mi...,def solution():\n #A circus has 12 clowns\n...,0,the Circus and Performing Arts
1025,A cheer leading squad has 20 members. \nThey h...,def solution():\n #A cheer leading squad ha...,800,cheer leading
667,3000 people attended a rally. 1000 of the atte...,def solution():\n #Total attendees\n tot...,0,unspecified
...,...,...,...,...
2644,A pterodactyl has 50 teeth. A tyrannosaurus re...,def solution():\n\n #A pterodactyl has 50 t...,25,unspecified
5736,Fairy Gardens has 1000 tulips. 200 of the tuli...,def solution():\n #Total tulips\n total_...,0,Fairy Gardens
889,25% of the 1200 students at a high school want...,def solution():\n #1200 students at a high ...,180.0,"Careers (e.g., firefighter, astronaut, doctor)"
664,Naruto ate 20 ramen noodles for lunch. He ate ...,def solution():\n #Naruto ate 20 ramen nood...,25,Naruto


In [12]:
df['topic'].value_counts()

topic
unspecified                                          3594
LeBron James                                          160
cheer leading                                         157
Captain America                                       156
Wonder Woman                                          155
                                                     ... 
Up                                                     10
Hobbies (e.g., stamp collecting, coin collecting)      10
Star Wars                                               9
Transformers                                            6
Pets (cats, dogs, hamsters)                             3
Name: count, Length: 198, dtype: int64